# Travel

In [20]:
import datetime
print(f"Last updated: {str(datetime.datetime.now())[:10]}")

Last updated: 2024-09-06


After traveling to various cultural sites for some years, I found a rule of thumb for finding good spots to visit: check if something is a UNESCO World Heritage site. Sometimes this leads to obvious suggestions (Vatican City, Paris, Amsterdam) but more often than not visiting world heritage sites has shown me sites that are very much worth visiting but not as touristy as the usual sites (e.g., Bologna, Orange, Albi).

At first, my enthousiasm led to the dream of visiting all world heritage sites in Europe, but this dream soon showed to be too ambitious. All EU countries then? Still quite a lot. Some narrowing down brought me to the following goal: visit all world heritage sites in the continental part of the original member states of the European Union. Achievable, but also ambitious, with Germany, France and Italy as the biggest obstacles. Since then, I've been using this as a heuristic in planning vacations.

(Of course, I still travel outside of these guidelines as well. Occasionally this leads to another world heritage site, and these are also included in the below map)

In [21]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import numpy as np
from travel_plotter import journey_generator

# Prepare dataset
def excel_fixer(input):
    if type(input) == float:
        return input
    else:
        return float(input.replace(',', '.'))
df = pd.read_csv("whc-sites-2023.csv", sep=';')
df['latitude'] = df['latitude'].apply(lambda x: excel_fixer(x))
df['longitude'] = df['longitude'].apply(lambda x: excel_fixer(x))
df['Year'] = df['Year'].apply(lambda x: x if not np.isnan(x) else "Not yet visited")
df['Rating'] = df['Rating'].apply(lambda x: excel_fixer(x))
df['Rating'] = df['Rating'].apply(lambda x: x if not np.isnan(x) else "Not yet visited")
df = df[df['Visited'].notna()]

fig = go.Figure()
for lats, lons, color, year in journey_generator(df):
    fig2 = px.line_mapbox(lat=lats, lon=lons, color_discrete_sequence=[color])
    fig2.update_layout(mapbox_style='carto-positron', hovermode=False)
    fig = go.Figure(data=fig.data + fig2.data)

max_bound = max(abs(min(df['latitude'].astype(float))-max(df['latitude'])), abs(min(df['longitude'])-max(df['longitude']))) * 175
zoom = 11.5 - np.log(max_bound)
fig1 = px.scatter_mapbox(df, lat='latitude', lon='longitude', color='Visited',
                        hover_name='name_en', mapbox_style='carto-positron', zoom=zoom,
                        custom_data=['Rating', 'Year'], hover_data={
                            'latitude':False,
                            'longitude':False,
                        })

fig1.update_traces(hovertemplate='<b>%{hovertext}</b><br>Rating: %{customdata[0]} <br>Visited in: %{customdata[1]}<extra></extra>')
fig1.update_layout(showlegend=False, margin={'t':0,'l':0,'b':0,'r':0})

fig = go.Figure(data=fig.data + fig1.data, layout=fig1.layout)

config = {'scrollZoom': True}
fig.show(config=config)


TypeError: scatter_mapbox() got an unexpected keyword argument 'scrollZoom'

(Lines mark seperate vacations. Lines in grey mark planned vacations, lines in light gray are trips I'd like to make someday)

In [ ]:
print("Progress:")
score = len(df[(df['Visited']) & (df['Outside EEG'] != True)]) / len(df)
displays = int(score * 40)
print(f"Total: {' ' * (20 - len('Total'))}[{'+' * displays}{'.' * (40 - displays)}] {len(df[(df['Visited']) & (df['Outside EEG'] != True)])} out of {len(df[df['Outside EEG'] != True])}\n")
for abbr, country in zip(['nld', 'fra', 'ita', 'bel', 'deu', 'lux'], ["Netherlands", "France", "Italy", "Belgium", "Germany", "Luxembourg"]):
    score = len(df[(df['Visited']) & (df['udnp_code'].str.contains(abbr))]) / len(df[df['udnp_code'].str.contains(abbr)])
    displays = int(score * 40)
    symbol = "⋆" if score == 1 else "+"
    print(f"{country}: {' ' * (20 - len(country))}[{symbol * displays}{'.' * (40 - displays)}] {len(df[(df['Visited']) & (df['udnp_code'].str.contains(abbr))])} out of {len(df[df['udnp_code'].str.contains(abbr)])}")

print("\nNote: due to transboundary sites the sum of all countries might not match the total")

Progress:
Total:                [+++++++++...............................] 46 out of 176

Netherlands:          [++++++++++++++++++++....................] 6 out of 12
France:               [++++++++++++............................] 15 out of 47
Italy:                [++++++..................................] 9 out of 60
Belgium:              [+++++++++++++++++++++++++...............] 10 out of 16
Germany:              [++++++++++..............................] 14 out of 54
Luxembourg:           [........................................] 0 out of 1

Note: due to transboundary sites the sum of all countries might not match the total
